In [1]:
import os
print(os.getcwd())

d:\Develops\ML_PROJECT\NER-PROJECT\notebook


In [2]:
import os
os.makedirs('data/ultra_fine', exist_ok=True)

In [3]:
import datasets

d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import load_dataset

# Load WNUT 2017 NER dataset
dataset = load_dataset("wnut_17")
print(dataset)


d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jotir\.cache\huggingface\hub\datasets--wnut_17. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate dev

RuntimeError: Dataset scripts are no longer supported, but found wnut_17.py

In [1]:
import os
import json
from typing import List, Dict, Any, Tuple
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer


class EntityTypingDataset(Dataset):
    """
    Dataset class for FIGER / Ultra-Fine Entity Typing.
    Each example = (context, mention, labels).
    """
    def __init__(self, file_path: str, label_map: Dict[str, int],
                 tokenizer_name: str, max_len: int = 128):
        self.samples = self._load_data(file_path)
        self.label_map = label_map
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_len = max_len

    def _load_data(self, file_path: str) -> List[Dict[str, Any]]:
        """Load dataset from JSON lines."""
        samples = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                example = json.loads(line.strip())
                samples.append(example)
        return samples

    def _labels_to_vector(self, labels: List[str]):
        """Convert list of labels into multi-hot vector."""
        vec = [0] * len(self.label_map)
        for l in labels:
            if l in self.label_map:
                vec[self.label_map[l]] = 1
        return vec

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        mention = sample["mention"]
        context = sample["context"]
        labels = sample["labels"]

        # Highlight mention inside context
        text = context.replace(mention, f"[MENTION] {mention} [/MENTION]")

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        label_vec = self._labels_to_vector(labels)

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": label_vec,
            "mention": mention,
            "context": context
        }


def build_label_map(files: List[str]) -> Dict[str, int]:
    """
    Automatically build label -> index mapping from dataset files.
    Args:
        files: list of file paths (train/dev/test).
    Returns:
        label_map: dict mapping label string -> index
    """
    unique_labels = set()
    for file_path in files:
        if not os.path.exists(file_path):
            continue
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                example = json.loads(line.strip())
                for l in example.get("labels", []):
                    unique_labels.add(l)

    label_map = {label: idx for idx, label in enumerate(sorted(unique_labels))}
    return label_map


def get_dataloader(file_path: str, label_map: Dict[str, int],
                   tokenizer_name: str, batch_size: int = 16,
                   shuffle: bool = True, max_len: int = 128):
    dataset = EntityTypingDataset(file_path, label_map, tokenizer_name, max_len)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#from src.NER.components.data_loader import get_dataloader, build_label_map

# Build label_map automatically from dataset files
label_map = build_label_map([
    "data/ultra_fine/train.json",
    "data/ultra_fine/dev.json",
    "data/ultra_fine/test.json"
])

print(f"Total labels found: {len(label_map)}")
print(list(label_map.items())[:10])  # show first 10 labels

# Create train loader
train_loader = get_dataloader(
    file_path="data/ultra_fine/train.json",
    label_map=label_map,
    tokenizer_name="bert-base-uncased",
    batch_size=32
)

# Inspect one batch
batch = next(iter(train_loader))
print(batch["input_ids"].shape)   # (batch_size, max_len)
print(batch["labels"][0])         # multi-hot label vector


Total labels found: 0
[]


FileNotFoundError: [Errno 2] No such file or directory: 'data/ultra_fine/train.json'

In [4]:
import os
import requests
import zipfile
import tarfile

FIGER_URL = "https://github.com/xiaoling/figer/archive/refs/heads/master.zip"
ULTRA_FINE_URL = "https://github.com/uwnlp/open_type/archive/refs/heads/master.zip"

DATA_DIR = "data"

def download_file(url, save_path):
    """Download a file from a URL."""
    if os.path.exists(save_path):
        print(f"✅ File already exists: {save_path}")
        return
    print(f"⬇️ Downloading {url} ...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(save_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"✅ Saved: {save_path}")

In [5]:
figer_zip = os.path.join(DATA_DIR, "figer.zip")
download_file(FIGER_URL, figer_zip)

⬇️ Downloading https://github.com/xiaoling/figer/archive/refs/heads/master.zip ...
✅ Saved: data\figer.zip


In [6]:
# Ultra-Fine
ultra_zip = os.path.join(DATA_DIR, "ultra_fine.zip")
download_file(ULTRA_FINE_URL, ultra_zip)
#extract_file(ultra_zip, os.path.join(DATA_DIR, "ultra_fine"))



⬇️ Downloading https://github.com/uwnlp/open_type/archive/refs/heads/master.zip ...
✅ Saved: data\ultra_fine.zip


In [7]:
def extract_file(file_path, extract_to):
    """Extract ZIP or TAR files."""
    print(f"📦 Extracting {file_path} ...")
    if file_path.endswith(".zip"):
        with zipfile.ZipFile(file_path, "r") as zf:
            zf.extractall(extract_to)
    elif file_path.endswith((".tar.gz", ".tgz")):
        with tarfile.open(file_path, "r:gz") as tf:
            tf.extractall(extract_to)
    print(f"✅ Extracted to {extract_to}")

In [8]:
extract_file(figer_zip, os.path.join(DATA_DIR, "figer"))

📦 Extracting data\figer.zip ...
✅ Extracted to data\figer


In [9]:
extract_file(ultra_zip, os.path.join(DATA_DIR, "ultra_fine"))

📦 Extracting data\ultra_fine.zip ...
✅ Extracted to data\ultra_fine


In [7]:
def convert_figer(input_file: str, output_file: str):
    """
    Convert FIGER TSV format → JSONL format.
    Expected FIGER format per line:
        sentence<TAB>mention<TAB>start<TAB>end<TAB>labels(comma-separated)

    Output format:
        {"mention": ..., "context": ..., "labels": [...]}
    """
    print(f"📂 Converting FIGER: {input_file} → {output_file}")
    with open(input_file, "r", encoding="utf-8") as fin, \
         open(output_file, "w", encoding="utf-8") as fout:
        for line in fin:
            parts = line.strip().split("\t")
            if len(parts) < 5:
                continue
            sentence, mention, start, end, labels = parts
            labels = labels.split(",")
            example = {
                "mention": mention,
                "context": sentence,
                "labels": labels
            }
            fout.write(json.dumps(example) + "\n")
    print(f"✅ Saved JSONL: {output_file}")

In [8]:
def prepare_jsonl_datasets():
    """
    Convert raw FIGER / Ultra-Fine into clean JSONL files
    ready for data_loader.py
    """
    data_dir = "data"

    # FIGER paths (you may need to update based on repo structure)
    figer_dir = os.path.join(data_dir, "figer", "figer-master", "data")
    if os.path.exists(figer_dir):
        convert_figer(os.path.join(figer_dir, "train.tsv"), os.path.join(data_dir, "figer_train.json"))
        convert_figer(os.path.join(figer_dir, "dev.tsv"), os.path.join(data_dir, "figer_dev.json"))
        convert_figer(os.path.join(figer_dir, "test.tsv"), os.path.join(data_dir, "figer_test.json"))


In [15]:
#!curl http://nlp.cs.washington.edu/figer_data.tar.gz -O data/figer_data.tar.gz
!tar -xvzf figer_data.tar.gz -C data/


tar: Error opening archive: Unrecognized archive format


In [ ]:
tar: Error opening archive: Unrecognized archive format

In [16]:
from datasets import load_dataset

dataset = load_dataset("figer", "default")
print(dataset)


d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetNotFoundError: Dataset 'figer' doesn't exist on the Hub or cannot be accessed.

In [1]:
!7z x .\figer_data.tar.gz

'7z' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import tarfile
with tarfile.open("figer_data.tar.gz", "r:gz") as tar:
    tar.extractall("data/")

ReadError: not a gzip file

In [12]:
import os
import json
import urllib.request
import tarfile
from pathlib import Path
from typing import List, Dict

def download_ultrafine_data(output_dir="data/ultra_fine"):
    """
    Download and extract the Ultra-Fine Entity Typing dataset.
    """
    os.makedirs(output_dir, exist_ok=True)
    tar_path = os.path.join(output_dir, "ultrafine.tar.gz")

    url = "https://nlp.cs.washington.edu/entity_type_data/ultrafine.tar.gz"

    if not os.path.exists(tar_path):
        print(f"📥 Downloading Ultra-Fine dataset from {url}...")
        urllib.request.urlretrieve(url, tar_path)
        print("✅ Download complete.")

    # Extract
    extracted_dir = os.path.join(output_dir, "ultrafine")
    if not os.path.exists(extracted_dir):
        print("📦 Extracting dataset...")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=output_dir)
        print("✅ Extraction complete.")
    else:
        print("ℹ️ Already extracted.")

    return extracted_dir


def convert_ultrafine_to_jsonl(raw_dir: str, save_dir: str):
    """
    Convert raw JSON format into clean JSONL (line-by-line JSON) format.
    """
    os.makedirs(save_dir, exist_ok=True)

    for split in ["train", "dev", "test"]:
        src_path = os.path.join(raw_dir, f"{split}.json")
        dst_path = os.path.join(save_dir, f"{split}.jsonl")

        if not os.path.exists(src_path):
            print(f"⚠️ Missing file: {src_path}")
            continue

        with open(src_path, encoding="utf-8") as f:
            data = json.load(f)

        with open(dst_path, "w", encoding="utf-8") as out:
            for ex in data:
                out.write(json.dumps(ex) + "\n")

        print(f"✅ Converted {split}: {len(data)} samples saved to {dst_path}")


def prepare_ultrafine_dataset():
    """
    Full pipeline: download, extract, and convert Ultra-Fine dataset.
    """
    raw_dir = download_ultrafine_data()
    save_dir = "data/ultra_fine_ready"
    convert_ultrafine_to_jsonl(raw_dir, save_dir)
    print("🎯 Ultra-Fine dataset ready for loading.")
    return save_dir


#if __name__ == "__main__":
prepare_ultrafine_dataset()


📥 Downloading Ultra-Fine dataset from https://nlp.cs.washington.edu/entity_type_data/ultrafine.tar.gz...


HTTPError: HTTP Error 404: Not Found

In [3]:
import os
import json
from datasets import load_dataset

d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def prepare_fewnerd_dataset(save_dir="data/fewnerd", subset="supervised"):
    """
    Auto-download Few-NERD dataset and build label map.
    """
    print(f"📥 Downloading Few-NERD ({subset})...")
    dataset = load_dataset("ZihanWangKiwi/few-nerd", subset)

    os.makedirs(save_dir, exist_ok=True)

    label_set = set()

In [7]:
prepare_fewnerd_dataset()

📥 Downloading Few-NERD (supervised)...


DatasetNotFoundError: Dataset 'ZihanWangKiwi/few-nerd' doesn't exist on the Hub or cannot be accessed.

In [9]:
# Python
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "train.json", "test": "test.json"})

FileNotFoundError: Unable to find 'D:/Develops/ML_PROJECT/NER-PROJECT/notebook\train.json'

In [10]:
dataset = load_dataset("DFKI-SLT/few-nerd", subset = "supervised")

RuntimeError: Dataset scripts are no longer supported, but found few-nerd.py

In [ ]:
!wget -O data_dir/supervised.zip https://cloud.tsinghua.edu.cn/f/c1f71c011d6b461786bc/?dl=1

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!curl -O $data_dir/supervised.zip https://cloud.tsinghua.edu.cn/f/c1f71c011d6b461786bc/?dl=1

curl: Failed to extract a filename from the URL to use for storage
curl: (3) URL using bad/illegal format or missing URL


In [23]:
import wget
import requests

In [25]:
url = "https://cloud.tsinghua.edu.cn/f/c1f71c011d6b461786bc/?dl=supervised"
save_path = "data"

In [18]:
downloaded_file = wget.download(url)

In [19]:
download_file

NameError: name 'download_file' is not defined

In [26]:
# Python
def download_file(url, save_path):
    """Download a file from a URL."""
    if os.path.exists(save_path):
        print(f"✅ File already exists: {save_path}")
        return
    print(f"⬇️ Downloading {url} ...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(save_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"✅ Saved: {save_path}")

In [27]:
download_file(url, save_path)

⬇️ Downloading https://cloud.tsinghua.edu.cn/f/c1f71c011d6b461786bc/?dl=supervised ...
✅ Saved: data
